In [1]:
# In this script, we will read in the surface air temp from CMIP5 output,
# calculate the change in SAT relative to the control,
# and quantify the polar amplification ratio.

# By: Ty Janoski
# updated: 08.06.20

In [2]:
# Import statements

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [3]:
def spatial_mean(ds,lat_bound=-90):
    """Take the mean over latitude and longitude dimensions,
    while weighting for latitude.

    PARAMETERS
    ----------
    datasets : list of xarray.Dataset
        A list of xarray datasets to return .

    RETURNS
    -------
    filelist : A list of xarray datasets retrieved from server.
    """
    area_of_int = ds.mean(dim='lon').sel(lat = slice(lat_bound, None))
    weights = (np.cos(np.deg2rad(area_of_int['lat']))/
               np.sum(np.cos(np.deg2rad(area_of_int['lat']))))
    return((area_of_int * weights).sum(dim='lat'))

In [4]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR','GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2', 'bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
glb_list = []
arc_list = []
for mod in models:
    print(mod)
    
    # read in last 50 years of piControl
    picontrol = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/tas_Amon_'+mod+'_*.nc',
                              parallel=True,combine='by_coords',use_cftime=True).tas.isel(time=
                                                                          slice(-600,None))

    # make monthly climatology
    clim = picontrol.groupby(picontrol.time.dt.month).mean(dim='time')

    # read in first 150 years of abrupt4xCO2
    abr = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/abrupt4xCO2/tas_Amon_'+mod+'_*.nc',
                                  parallel=True,combine='by_coords',use_cftime=True).tas.isel(time=
                                                                              slice(None,1800))

    # calculate difference between abrupt4xCO2 and clim (clim must be tiled to be 150 years long)
    diff = abr - np.tile(clim,(150,1,1))

    # spatial average difference to get global and arctic ΔSAT
    # remove calendar because all models are on different calendars
    glb = spatial_mean(diff).compute()
    glb['time'] = np.arange(0,1800,1)
    glb['year'] = (glb.time // 12) + 1
    arc = spatial_mean(diff,lat_bound=70).compute()
    arc['time'] = np.arange(0,1800,1)
    arc['year'] = (arc.time // 12) + 1
    
    # append to list
    glb_list.append(glb)
    arc_list.append(arc)

ACCESS1-0
ACCESS1-3
CNRM-CM5
IPSL-CM5B-LR
GFDL-ESM2G
MIROC-ESM
FGOALS-g2
bcc-csm1-1
BNU-ESM
CanESM2
CCSM4
CSIRO-Mk3-6-0
FGOALS-s2
GFDL-CM3
inmcm4
IPSL-CM5A-LR
MIROC5
MPI-ESM-LR
MPI-ESM-P
MRI-CGCM3
NorESM1-M


In [5]:
glb_da = xr.concat(glb_list,dim='model')
arc_da = xr.concat(arc_list,dim='model')

In [6]:
for m in range(len(models)):
    print(models[m]+': '+str(np.round(float(arc_da.isel(model=m,time=0)),2))+'K')

ACCESS1-0: -2.03K
ACCESS1-3: 1.93K
CNRM-CM5: 4.24K
IPSL-CM5B-LR: 3.59K
GFDL-ESM2G: -3.15K
MIROC-ESM: 0.86K
FGOALS-g2: 1.29K
bcc-csm1-1: -2.72K
BNU-ESM: 4.56K
CanESM2: 0.19K
CCSM4: 1.95K
CSIRO-Mk3-6-0: 1.57K
FGOALS-s2: 3.68K
GFDL-CM3: -2.94K
inmcm4: 1.95K
IPSL-CM5A-LR: 1.1K
MIROC5: 2.34K
MPI-ESM-LR: -0.86K
MPI-ESM-P: 2.64K
MRI-CGCM3: 0.65K
NorESM1-M: -1.37K


In [7]:
fig, axes = plt.subplots(ncols=2,figsize=(7.4,3),sharey=True)

# show monthly change in SAT
arc_mon = arc_da.isel(time=slice(0,30))
glb_mon = glb_da.isel(time=slice(0,30))

# get annual average
arc_ann = arc_da.groupby('year').mean(dim='time')
glb_ann = glb_da.groupby('year').mean(dim='time')

# plot away!
axes[0].plot(arc_mon.time+1,arc_mon.mean(dim='model'),color='red',label='Arctic')
axes[0].fill_between(arc_mon.time+1,arc_mon.mean(dim='model')-arc_mon.std(dim='model'),
                      arc_mon.mean(dim='model')+arc_mon.std(dim='model'),color='red',alpha=0.4)

axes[0].plot(glb_mon.time+1,glb_mon.mean(dim='model'),color='navy',label='global')
axes[0].fill_between(glb_mon.time+1,glb_mon.mean(dim='model')-glb_mon.std(dim='model'),
                      glb_mon.mean(dim='model')+glb_mon.std(dim='model'),color='navy',alpha=0.4)

axes[0].set_xlim([1,30])
axes[0].set_xticks(np.arange(3,31,3))
axes[0].yaxis.set_tick_params(which='both', labelleft=True)
axes[0].tick_params(labelsize=8)
axes[0].set_ylabel('$\Delta$SAT relative to control (K)',fontsize=8)
axes[0].set_xlabel('Months since 4xCO$_2$',fontsize=8)
axes[0].grid()
axes[0].legend(loc='upper left',fontsize=8)
axes[0].set_yticks(np.arange(0,18,2))
axes[0].set_title('(a)',fontsize=8)
axes[1].set_title('(b)',fontsize=8)

axes[1].plot(arc_ann.year,arc_ann.mean(dim='model'),color='red',label='Arctic')
axes[1].fill_between(arc_ann.year,arc_ann.mean(dim='model')-arc_ann.std(dim='model'),
                      arc_ann.mean(dim='model')+arc_ann.std(dim='model'),color='red',alpha=0.4)

axes[1].plot(glb_ann.year,glb_ann.mean(dim='model'),color='navy',label='global')
axes[1].fill_between(glb_ann.year,glb_ann.mean(dim='model')-glb_ann.std(dim='model'),
                      glb_ann.mean(dim='model')+glb_ann.std(dim='model'),color='navy',alpha=0.4)

axes[1].set_xlim([1,150])
axes[1].tick_params(labelsize=8)
axes[1].set_xlabel('Years since 4xCO$_2$',fontsize=8)
axes[1].grid()

plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/Fig_1_060820.pdf')

<Figure size 532.8x216 with 2 Axes>

In [8]:
# let's see at what point Arctic - global is significantly different than 0
# take difference between Arctic and global

diff_da = arc_da - glb_da

# we need mean and std dev for t-test
mean = diff_da.mean(dim='model')
std = diff_da.std(dim='model')
tvals = mean / (std / np.sqrt(len(diff_da.model)))

In [9]:
diff = diff_da.isel(time=slice(0,12))
plt.plot(diff.time+1,diff.mean(dim='model'),linestyle=':',color='navy')
plt.plot(diff.time+1,diff.mean(dim='model').where(tvals.isel(time=slice(0,12))>=2.086),color='navy')
plt.xticks(np.arange(1,13,1))
plt.xlim([1,12])
plt.grid()
plt.xlabel(r'Months since 4xCO$_2$')
plt.ylabel(r'$\Delta$SAT$_{Arctic-Global}$',fontsize=8)
plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/arc_minus_glb_052520.pdf')

<Figure size 432x288 with 1 Axes>

In [10]:
diff = diff_da.groupby('year').mean(dim='time')
# we need mean and std dev for t-test
mean = diff.mean(dim='model')
std = diff.std(dim='model')
tvals = mean / (std / np.sqrt(len(diff.model)))

plt.plot(diff.year,diff.mean(dim='model'),linestyle=':',color='navy')
plt.plot(diff.year,diff.mean(dim='model').where(tvals>=2.082),color='navy')
plt.fill_between(diff.year,mean-std,
                      mean+std,color='navy',alpha=0.4)
plt.xlim([1,150])
plt.grid()
plt.xlabel(r'Years since 4xCO$_2$',fontsize=8)
plt.ylabel(r'$\Delta$SAT$_{Arctic-Global}$',fontsize=8)
plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/arc_minus_glb_ann_060320.pdf')

<Figure size 432x288 with 1 Axes>

In [11]:
cummean = diff_da.cumsum(dim='time')/np.arange(1,1801,1)

In [12]:
# we need mean and std dev for t-test
mean = cummean.mean(dim='model')
std = cummean.std(dim='model')
tvals = mean / (std / np.sqrt(len(diff_da.model)))

In [13]:
float(tvals.isel(time=2))

2.6426156453119227

In [14]:
diff = cummean.isel(time=slice(0,12))
plt.plot(diff.time+1,diff.mean(dim='model'),linestyle=':',color='navy')
plt.plot(diff.time+1,diff.mean(dim='model').where(tvals.isel(time=slice(0,12))>=2.086),color='navy')
plt.xticks(np.arange(1,13,1))
plt.xlim([1,12])
plt.grid()
plt.xlabel(r'Months since 4xCO$_2$')
plt.ylabel(r'Cumulative Average $\Delta$SAT$_{Arctic-Global}$',fontsize=8)
plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/cum_avg_arc_minus_glb_052520.pdf')

<Figure size 432x288 with 1 Axes>

In [15]:
# define the polar amplification factor (PAF) as arctic/global SAT change
PAF_ann = arc_da.groupby('year').mean(dim='time') / (glb_da.groupby('year').mean(dim='time'))

In [16]:
# let's plot it

fig, ax = plt.subplots(figsize=(4.5,3))

# plot away!
ax.plot(PAF_ann.year,PAF_ann.mean(dim='model'),color='black')
ax.fill_between(PAF_ann.year,PAF_ann.mean(dim='model')-PAF_ann.std(dim='model'),
                      PAF_ann.mean(dim='model')+PAF_ann.std(dim='model'),color='black',alpha=0.4)

ax.set_xlim([1,150])
ax.set_ylim([1,3.3])
ax.tick_params(labelsize=8)
ax.set_ylabel('PAF',fontsize=8)
ax.set_xlabel('Years since 4xCO$_2$',fontsize=8)
ax.grid()
# axes[0].set_yticks(np.arange(0,18,2))

plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/Fig_2_060820.svg')

<Figure size 324x216 with 1 Axes>

In [33]:
fig,axes = plt.subplots(nrows=3,figsize=(3.7,6))

# plot away!
axes[0].plot(arc_mon.time+1,arc_mon.mean(dim='model'),color='red',label='Arctic')
axes[0].fill_between(arc_mon.time+1,arc_mon.mean(dim='model')-arc_mon.std(dim='model'),
                      arc_mon.mean(dim='model')+arc_mon.std(dim='model'),color='red',alpha=0.4)

axes[0].plot(glb_mon.time+1,glb_mon.mean(dim='model'),color='navy',label='global')
axes[0].fill_between(glb_mon.time+1,glb_mon.mean(dim='model')-glb_mon.std(dim='model'),
                      glb_mon.mean(dim='model')+glb_mon.std(dim='model'),color='navy',alpha=0.4)

axes[0].set_xlim([1,30])
axes[0].set_xticks(np.arange(3,31,3))
axes[0].yaxis.set_tick_params(which='both', labelleft=True)
axes[1].yaxis.set_tick_params(which='both', labelleft=True)
axes[0].tick_params(labelsize=8)
axes[0].set_ylabel('$\Delta$SAT (K)',fontsize=8)
axes[1].set_ylabel('$\Delta$SAT (K)',fontsize=8)
axes[0].set_xlabel('Months since 4xCO$_2$',fontsize=8)
axes[0].grid()
axes[0].legend(loc='upper left',fontsize=8)
axes[0].set_yticks(np.arange(0,18,2))
axes[1].set_yticks(np.arange(0,18,2))
axes[0].set_title('(a)',fontsize=8)
axes[1].set_title('(b)',fontsize=8)
axes[2].set_title('(c)',fontsize=8)

axes[1].plot(arc_ann.year,arc_ann.mean(dim='model'),color='red',label='Arctic')
axes[1].fill_between(arc_ann.year,arc_ann.mean(dim='model')-arc_ann.std(dim='model'),
                      arc_ann.mean(dim='model')+arc_ann.std(dim='model'),color='red',alpha=0.4)

axes[1].plot(glb_ann.year,glb_ann.mean(dim='model'),color='navy',label='global')
axes[1].fill_between(glb_ann.year,glb_ann.mean(dim='model')-glb_ann.std(dim='model'),
                      glb_ann.mean(dim='model')+glb_ann.std(dim='model'),color='navy',alpha=0.4)

axes[1].set_xlim([1,150])
axes[1].tick_params(labelsize=8)
axes[1].set_xlabel('Years since 4xCO$_2$',fontsize=8)
axes[1].grid()

# plot away!
axes[2].plot(PAF_ann.year,PAF_ann.mean(dim='model'),color='black')
axes[2].fill_between(PAF_ann.year,PAF_ann.mean(dim='model')-PAF_ann.std(dim='model'),
                      PAF_ann.mean(dim='model')+PAF_ann.std(dim='model'),color='black',alpha=0.4)

axes[2].set_xlim([1,150])
axes[2].set_ylim([1,3.3])
axes[2].tick_params(labelsize=8)
axes[2].set_ylabel('PAF',fontsize=8)
axes[2].set_xlabel('Years since 4xCO$_2$',fontsize=8)
axes[2].grid()

plt.tight_layout(h_pad=0.01,w_pad=0.01)
# plt.show()
plt.savefig('/home/tylerj/CMIP5_AA_final/Fig_1_080620.svg')

<Figure size 266.4x432 with 3 Axes>